In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import shutil
import IPython
import IPython.display as ipd
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import librosa
from scipy.io import wavfile
from tqdm import tqdm_notebook
from sklearn.cross_validation import StratifiedKFold
from pathlib import Path
import torch
from fastai import *
from fastai.dataset import *
from fastai.learner import *
from fastai.text import *
from shutil import copyfile
from operator import itemgetter


%matplotlib inline
matplotlib.style.use('ggplot')
# torch.cuda.set_device(0)

/Users/blakewest/anaconda/envs/fastaudio/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
torch.version.__version__

'0.4.0'

In [3]:
DATA_PATH = Path('../../fastai/courses/dl2/data/freesound/')
TRAIN_PATH = DATA_PATH/'audio_train_sample'
TEST_PATH = DATA_PATH/'audio_test'
LABEL_PATH = DATA_PATH/'train_sample.csv'

In [4]:
train = pd.read_csv(LABEL_PATH)

In [37]:
class BasicAudioModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv1d(1, 16, 9),
            nn.ReLU(),
            nn.Conv1d(16, 16, 9),
            nn.ReLU(),
            nn.MaxPool1d(16),
            nn.Dropout(0.1),
            nn.Conv1d(16, 32, 3),
            nn.ReLU(),
            nn.Conv1d(32, 32, 3),
            nn.ReLU(),
            nn.MaxPool1d(8),
            nn.Dropout(0.01),
            nn.Conv1d(32, 256, 3),
            nn.ReLU(),
            nn.Conv1d(256, 256, 3),
            nn.Dropout(0.01),
            nn.MaxPool1d(4),
            nn.Conv1d(256, num_classes, 3),
            Lambda(lambda tensor: torch.mean(tensor, 2, keepdim=True)),
            Lambda(lambda tensor: tensor.view(tensor.shape[0], -1)),
        )

    def forward(self, input):
        return self.model(input)
    
class ResLayer(nn.Module):
    def __init__(self, num_filters, dilation, kernel_size=3):
        super().__init__()
        # This should return an output of equal size.
        self.conv_model = nn.Sequential(
            nn.Conv1d(num_filters, num_filters, kernel_size=kernel_size, dilation=dilation, bias=False, padding=kernel_size // 2),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(num_filters, momentum=0.01),
            nn.Conv1d(num_filters, num_filters, 1)
        )
        
    def forward(self, x):
        convolved = self.conv_model(x)
        # Dilation lop off like (n=dilation) samples from the end of the input, which means you can't
        # add the res block. So the padding fixes that, and lets us do the residual addition.
        padding = nn.ReplicationPad1d((0, x.size()[-1] - convolved.size()[-1]))
        return x.add(padding(convolved))
    
class AudioResBlock(nn.Module):
    def __init__(self, num_filters, num_layers):
        super().__init__()
        dilation = 1
        layers = []
        for layer in range(num_layers):
            cur_dilation = dilation * (2 ** layer)
            layers += [ResLayer(num_filters, cur_dilation)]
            
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)
    
class WavenetEncoder(nn.Module):
    def __init__(self, num_classes, num_filters=64, num_layers_per_res_block=1, bottleneck_filters=64, bs=16):
        super().__init__()
        self.num_filters = num_filters
        self.init_conv = nn.Conv1d(1, num_filters, 1, bias=False)
        self.temporal_encoder = self.get_temporal_encoder(num_filters, num_layers_per_res_block)
        self.bottleneck = nn.Conv1d(num_filters, bottleneck_filters, 1)
        self.classifier = self.get_classifier(bottleneck_filters, num_classes)
        
    def forward(self, x):
        return self.classifier(self.encode(x))
    
    def encode(self, x):
        x = self.init_conv(x)
        x = self.temporal_encoder(x)
        return self.bottleneck(x)
    
    def get_temporal_encoder(self, num_filters, num_layers):
        return AudioResBlock(num_filters, num_layers)
    
    def get_classifier(self, num_ch_in, num_classes):
        return nn.Sequential(
            # This is averagies each channel to a resolution 512 samples, or ~32ms. Taken from https://arxiv.org/pdf/1704.01279.pdf
            nn.AvgPool1d(512),
            # Create a convolution filter for each possible class
            nn.Conv1d(num_ch_in, num_classes, 1),
            # Averaging each classes across time, so you get 1 score for the whole sample for each classes
            Lambda(lambda tensor: torch.mean(tensor, 2, keepdim=True)),
            # Turn into a 1D tensor with a score for each class. PyTorch will auto softmax this if using Cross Ent Loss
            Lambda(lambda tensor: tensor.view(tensor.shape[0], -1)),
        )
        
        
# def _encode(self,sample):
#     sample = self.en_causal_layer(sample)

#     for i,(dilation_layer,dense_layer) in enumerate(zip(self.en_dilation_layer_stack,self.en_dense_layer_stack)):

#         current = sample

#         sample = F.relu(sample)
#         sample = dilation_layer(sample)
#         sample = F.relu(sample)
#         dense_layer is 1x1 conv
#         sample = dense_layer(sample)
#         _,_,current_length = sample.size()
#         current_in_sliced = current[:,:,-current_length:]
#         sample = sample + current_in_sliced

#     sample = self.bottleneck_layer(sample)  
#    # print(sample.size())  
#     pool1d = nn.AvgPool1d(self.en_pool_kernel_size)
#     sample = pool1d(sample)
#     return sample

def get_trn_val_split(x, y, val_pct=0.15):
    val_idxs = get_cv_idxs(len(x), val_pct=val_pct)
    if isinstance(x, list):
        return [([arr[i] for i in val_idxs], [arr[i] for i in range(len(arr)) if i not in val_idxs]) for arr in [x,y]]
    else:
        return split_by_idx(val_idxs, x, y)

class AudioLearner(Learner):
    def __init__(self, data, models, **kwargs):
        super().__init__(data, models, **kwargs)

    def _get_crit(self, data):
        return F.cross_entropy

def load_audio_from_df(trn_path, trn_df, sample_rate=16000):
    return [retrieve_file(str(trn_path) + '/' + trn_df['fname'][i], sample_rate=sample_rate) for i in range(len(trn_df))]

def retrieve_file(filepath, sample_rate=16000):
    data, _ = librosa.core.load(filepath, sr=sample_rate, res_type='kaiser_fast')
    return data

def preprocess_audio(audio_files):
    norm = librosa.util.normalize
    trimmed_xs = [librosa.effects.trim(norm(x))[0] for x in audio_files]
    return [x.reshape(1, x.shape[0]) for x in trimmed_xs]

def preprocess_ys(labels, one_hot=False):
    if isinstance(labels[0], str):
        tok2int = {v:k for k,v in enumerate(np.unique(labels))}
        labels = np.array([tok2int[tok] for tok in labels])
    num_classes = len(np.unique(labels))
    if one_hot:
        return [one_hot(labels[i], num_classes).reshape(1, num_classes) for i in range(len(labels))]
    else:
        return labels

class AudioModelData():
    def __init__(self, path, trn_ds, val_ds, test_ds=None, bs=64, sample_rate=16000):
        self.path = path
        self.bs = bs
        self.trn_ds, self.val_ds, self.test_ds = trn_ds, val_ds, test_ds
        self.trn_dl = AudioDataLoader(trn_ds, bs, sampler=SortishSampler(trn_ds, key=lambda x: len(trn_ds[x][0][0]), bs=bs))
        self.val_dl = AudioDataLoader(val_ds, bs, sampler=SortishSampler(val_ds, key=lambda x: len(val_ds[x][0][0]), bs=bs))
        self.test_dl = AudioDataLoader(test_ds, bs, sampler=SortishSampler(test_ds, key=lambda x: len(test_ds[x][0][0]),bs=bs)) if test_ds is not None else None
        self.num_classes = self.trn_dl.dataset.get_c()
        self.sz = self.trn_ds.get_x(1).size

    @classmethod
    def from_path_and_dataframes(cls, trn_path, trn_df, test_path=None, test_df=None, val_path=None, val_df=None, val_pct=0.15, model_path='./', bs=64):
        xs = load_audio_from_df(trn_path, trn_df)
        xs = preprocess_audio(x)
        ys = preprocess_ys(trn_df['label'])
        if test_path is not None:
            text_xs = load_audio_from_df(test_path, test_df)
            text_xs = preprocess_audio(test_xs)
        else:
            test_x = None
        return cls.from_array(xs, ys, test_xs, bs=bs)

    @classmethod
    def from_array(cls, trn_x, trn_y, test_x=None, val_pct=0.15, bs=64, model_path="./", **kwargs):
        ((val_x, trn_x), (val_y, trn_y)) = get_trn_val_split(trn_x, trn_y, val_pct)
        trn_ds = AudioDataset(trn_x, trn_y)
        val_ds = AudioDataset(val_x, val_y)
        test_ds = AudioDataset(test_x, test_y) if test_x is not None else None
        return cls(model_path, trn_ds, val_ds, test_ds, bs=bs)

    def get_model(self, model_params={"type": "basic"}, optimizer=torch.optim.Adam):
        if model_params['type'] == "basic":
            model = BasicAudioModel(self.num_classes)
        elif model_params['type'] == "wavenet":
            model = WavenetEncoder(self.num_classes)
        else:
            raise "Unknown model type: " + model_params["type"]
        model = SingleModel(to_gpu(model))
        return AudioLearner(self, model, opt_fn=optimizer)

class AudioDataLoader(DataLoader):
    def get_batch(self, indexes):
        batch_data = [self.dataset[i] for i in indexes]
        x_lens = [len(item[0][0]) for item in batch_data]
        if len(np.unique(x_lens)) > 1:
            max_len = np.max(x_lens)
            for i, item in enumerate(batch_data):
                item = list(item)
                clip_len = len(item[0][0])
                item[0] = np.pad(item[0], ((0,0), (0, max_len-clip_len)), 'wrap')
                batch_data[i] = tuple(item)
        return self.np_collate(batch_data)
    
class AudioDataLoader(DataLoader):
    def get_batch(self, indexes):
        batch_data = [self.dataset[i] for i in indexes]
        x_lens = [item[0].shape[1] for item in batch_data]
        if len(np.unique(x_lens)) > 1:
            max_len = np.max(x_lens)
            for i, item in enumerate(batch_data):
                x, y = item
                clip_len = x.shape[1]
                pad_mode = 'wrap' if clip_len > 1 else 'constant'
                x = np.pad(x, ((0, 0), (0, max_len-clip_len)), pad_mode)
                batch_data[i] = x, y
        return self.np_collate(batch_data)

class AudioDataset(BaseDataset):
    def __init__(self, xs, ys, transforms=None):
        if isinstance(ys[0], str):
            ys = preprocess_ys(ys)
        self.ys = ys
        self.xs = xs
        assert(len(xs) == len(ys)), "Length of xs does not equal length of ys"
        super().__init__(transforms)

    def get_x(self, i):
        return self.xs[i]

    def get_y(self, i):
        return self.ys[i]

    def get_n(self):
        return len(self.xs)

    def get_sz(self):
        return self.get_x(1).shape[0]

    def get_c(self):
        return int(np.max(self.ys) + 1)

def create_sample_data(path, percent=0.1, sample_path=None, overwrite=False, labels_df=None):
    sample_path = path + '_sample' if sample_path is None else sample_path
    if not os.path.exists(sample_path):
            print("Creating folder for the sample set...", sample_path)
            os.mkdir(sample_path)
    existing_sample_files = glob(sample_path + '/*')
    if len(existing_sample_files) > 0:
        if not overwrite:
            print("Sample already exists. Pass overwrite=True to delete and redo")
            return
        else:
            for file in existing_sample_files:
                os.remove(file)
    print("Saving a", percent * 100, "percent sample to", sample_path)
    for filepath in glob(path + '/*'):
        if np.random.random() < percent:
            fname = filepath.split('/')[-1]
            copyfile(filepath, sample_path + '/' + fname)
    if labels_df:
        sample_fnames = [filepath.split('/')[-1] for filepath in glob(sample_path + '/*')]
        labels_df[labels_df['fname'].isin(sample_fnames)].to_csv(path + '../train_sample.csv')

def save_data(data, filename):
    with open(filename, 'wb') as outfile:
        pickle.dump(data, outfile, pickle.HIGHEST_PROTOCOL)

def load_data(filename):
    with open(filename, 'rb') as infile:
        result = pickle.load(infile)
    return result

def display_sample(train, category=None):
    sample = train[train['label'] == category].sample() if category else train.sample()
    fname = str(TRN_PATH/sample['fname'].values[0])
    print(sample)
    return ipd.Audio(fname)

def munge_and_save_data(trn_path, trn_df, x_filepath, y_filepath):
    print("Loading files...")
    xs = load_audio_from_df(trn_path, trn_df)
    print("Processing audio...")
    xs = preprocess_audio(xs)
    print("Processing labels...")
    ys = preprocess_ys(trn_df['label'])
    print("Saving xs and ys...")
    save_data(xs, x_filepath)
    save_data(ys, y_filepath)


In [6]:
# munge_and_save_data(TRAIN_PATH, train, x_filepath=DATA_PATH/'all_audio.pkl', y_filepath=DATA_PATH/'all_labels.pkl')

In [7]:
xs = load_data(DATA_PATH/'all_audio.pkl')
ys = load_data(DATA_PATH/'all_labels.pkl')

In [8]:
xs[0].size, ys[0].size

(188416, 1)

In [27]:
test_x = xs[2]
test_x = V(test_x).view(1, 1, -1)

In [28]:
test_x.size()

torch.Size([1, 1, 212480])

In [14]:
res_layer = ResLayer(256, 8, 8)
init_conv = nn.Conv1d(1, 256, 1)
test_model = nn.Sequential(init_conv, res_layer)

In [15]:
test = init_conv(test_x)
test = res_layer.conv_model(test)
padding = nn.ReplicationPad1d((0, test_x.size()[-1] - test.size()[-1]))
test = padding(test)
test.size(), test_x.size()

(torch.Size([1, 256, 64960]), torch.Size([1, 1, 64960]))

In [16]:
layer1 = test_model(test_x).size()

In [38]:
md = AudioModelData.from_array(xs[:16],ys[:16], bs=4)

In [39]:
learner = md.get_model(model_params={"type": "wavenet"})

In [36]:
learner.fit(lrs=0.001, n_cycle=1, metrics=[accuracy])

  0%|          | 0/4 [00:00<?, ?it/s]


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation

In [ ]:
# learner.fit(lrs=0.001, n_cycle=1, metrics=[accuracy])

  0%|          | 0/3 [00:00<?, ?it/s]